# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [126]:
# import libraries
import numpy as np
import pandas as pd

import sqlite3
from sqlalchemy import create_engine

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pickle

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anhtu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anhtu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anhtu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [127]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql("select * from disaster_messages", engine)

X = df['message']
Y = df.iloc[:, 4:]
X.head()
Y.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [128]:
category_counts = df.iloc[:,4:].sum().sort_values(ascending = False).values
category_counts
category_names = df.iloc[:,4:].sum().sort_values(ascending = False).index
category_names

array([20093, 10860,  7297,  5075,  4474,  3446,  2923,  2455,  2443,
        2314,  2155,  2084,  1705,  1672,  1376,  1333,  1313,  1201,
        1194,  1151,   875,   860,   724,   604,   532,   530,   471,
         405,   309,   298,   283,   282,   159,   120,   118,     0],
      dtype=int64)

Index(['related', 'aid_related', 'weather_related', 'direct_report', 'request',
       'other_aid', 'food', 'earthquake', 'storm', 'shelter', 'floods',
       'medical_help', 'infrastructure_related', 'water', 'other_weather',
       'buildings', 'medical_products', 'transport', 'death',
       'other_infrastructure', 'refugees', 'military', 'search_and_rescue',
       'money', 'electricity', 'cold', 'security', 'clothing', 'aid_centers',
       'missing_people', 'hospitals', 'fire', 'tools', 'shops', 'offer',
       'child_alone'],
      dtype='object')

In [129]:
df.iloc[:,4:].sum().sort_values(ascending = False)

related                   20093
aid_related               10860
weather_related            7297
direct_report              5075
request                    4474
other_aid                  3446
food                       2923
earthquake                 2455
storm                      2443
shelter                    2314
floods                     2155
medical_help               2084
infrastructure_related     1705
water                      1672
other_weather              1376
buildings                  1333
medical_products           1313
transport                  1201
death                      1194
other_infrastructure       1151
refugees                    875
military                    860
search_and_rescue           724
money                       604
electricity                 532
cold                        530
security                    471
clothing                    405
aid_centers                 309
missing_people              298
hospitals                   283
fire    

### 2. Write a tokenization function to process your text data

In [132]:

def tokenize(text):
    """
    The function tokenizes and lemmatizes the text in argument, then removes
    any characters not alphanumerical, then converts the tokens into lower characters.
    Afterwards, it removes english stop words.
    The function finally returns a list of tokens
    """
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    tokens = [lemmatizer.lemmatize(word).strip() for word in tokens if word not in stop_words]
       
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [133]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer(smooth_idf=False)),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [134]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002292D34D790>)),
                ('tfidf', TfidfTransformer(smooth_idf=False)),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [135]:
Y_pred = pipeline.predict(X_test)

In [136]:
category_names = Y_test.columns

In [137]:
def validate_model(Y_test,Y_pred):
    for column_index, column_name in enumerate(category_names):
        print(column_name, '\n',classification_report(Y_test[column_name], Y_pred[:, column_index]))
    accuracy_per_feature = (Y_pred == Y_test).mean()
    mean_accuracy = np.mean(accuracy_per_feature)
    print("Final mean accuracy : {:.5f}".format(mean_accuracy))

In [138]:
validate_model(Y_test,Y_pred)

related 
               precision    recall  f1-score   support

           0       0.54      0.53      0.54      1553
           1       0.86      0.86      0.86      5001

    accuracy                           0.78      6554
   macro avg       0.70      0.70      0.70      6554
weighted avg       0.78      0.78      0.78      6554

request 
               precision    recall  f1-score   support

           0       0.91      0.91      0.91      5480
           1       0.56      0.55      0.56      1074

    accuracy                           0.86      6554
   macro avg       0.74      0.73      0.73      6554
weighted avg       0.85      0.86      0.86      6554

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99   

### 6. Improve your model
Use grid search to find better parameters. 

In [139]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__random_state', 'clf__estimator__splitter', 'clf__estimator', 'clf__n_jobs'])

In [140]:
max_depth = [2,4]
parameters = dict(clf__estimator__max_depth=max_depth)

In [141]:
cv = GridSearchCV(pipeline,param_grid=parameters)
cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002292D34D790>)),
                                       ('tfidf',
                                        TfidfTransformer(smooth_idf=False)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=DecisionTreeClassifier()))]),
             param_grid={'clf__estimator__max_depth': [2, 4]})

In [142]:
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002292D34D790>)),
                                       ('tfidf',
                                        TfidfTransformer(smooth_idf=False)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=DecisionTreeClassifier()))]),
             param_grid={'clf__estimator__max_depth': [2, 4]})

In [143]:
cv.best_params_

{'clf__estimator__max_depth': 4}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [144]:
Y_pred = cv.predict(X_test)

In [145]:
validate_model(Y_test,Y_pred)

related 
               precision    recall  f1-score   support

           0       0.63      0.13      0.21      1553
           1       0.78      0.98      0.87      5001

    accuracy                           0.78      6554
   macro avg       0.71      0.55      0.54      6554
weighted avg       0.75      0.78      0.71      6554

request 
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      5480
           1       0.81      0.37      0.50      1074

    accuracy                           0.88      6554
   macro avg       0.85      0.67      0.72      6554
weighted avg       0.88      0.88      0.86      6554

offer 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99   

other_infrastructure 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6256
           1       0.47      0.03      0.05       298

    accuracy                           0.95      6554
   macro avg       0.71      0.51      0.51      6554
weighted avg       0.93      0.95      0.93      6554

weather_related 
               precision    recall  f1-score   support

           0       0.83      0.98      0.90      4769
           1       0.88      0.46      0.60      1785

    accuracy                           0.84      6554
   macro avg       0.86      0.72      0.75      6554
weighted avg       0.84      0.84      0.82      6554

floods 
               precision    recall  f1-score   support

           0       0.96      0.99      0.98      6022
           1       0.82      0.57      0.67       532

    accuracy                           0.95      6554
   macro avg       0.89      0.78      0.82      6554
weighted avg       0.95 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [98]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer(smooth_idf=False)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [106]:
parameters = {
        'clf__estimator__n_estimators': [100, 500]
}

In [100]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estim

In [104]:
cv_2 = GridSearchCV(pipeline, param_grid=parameters)
cv_2

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002291E051940>)),
                                       ('tfidf',
                                        TfidfTransformer(smooth_idf=False)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20]})

In [146]:
cv_2.fit(X_train, Y_train)

In [ ]:
cv_2.best_params_

In [ ]:
Y_pred = cv_2.predict(X_test)

In [ ]:
validate_model(Y_test,Y_pred)

### 9. Export your model as a pickle file

In [116]:
pickle.dump(cv, open('models/classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.